In [2]:
import pandas as pd 
import numpy as np 
import math 
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
import re

# MEN'S ANALYSIS

## Reading in the data

In [2]:
# Men's regular season detailed stats
MRegStats = pd.read_csv("Data/MRegularSeasonDetailedResults.csv")

# Men's regular season compact stats
MCompStats = pd.read_csv("Data/MRegularSeasonCompactResults.csv")

# Men's NCAA tournament detailed stats
MTournStats = pd.read_csv("Data/MNCAATourneyDetailedResults.csv")

# Men's NCAA tournament compact stats
MCompTournStats = pd.read_csv("Data/MNCAATourneyCompactResults.csv")

# Men's Conference tournament stats
MConfTournStats = pd.read_csv("Data/MConferenceTourneyGames.csv")

# Men's Team names
MTeams = pd.read_csv("Data/MTeams.csv")
MTeamSpellings = pd.read_csv("Data/MTeamSpellings.csv", encoding='unicode_escape')

# Men's Massey Ordinals
MOridinals = pd.read_csv("Data/MMasseyOrdinals.csv")

# Men's Tournament Seeds
MSeeds = pd.read_csv("Data/MNCAATourneySeeds.csv")

# Men's Coaches
MCoaches = pd.read_csv("Data/MTeamCoaches.csv")

# Men's Conferences
MConferences = pd.read_csv("Data/MTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
MGameCities = pd.read_csv("Data/MGameCities.csv")

In [3]:
# Basic Percentage Stats
# FG Perc
MRegStats["WFGPerc"] = MRegStats["WFGM"] / MRegStats["WFGA"]
MRegStats["LFGPerc"] = MRegStats["LFGM"] / MRegStats["LFGA"]

# 3PT Perc
MRegStats["WFG3Perc"] = MRegStats["WFGM3"] / MRegStats["WFGA3"]
MRegStats["LFG3Perc"] = MRegStats["LFGM3"] / MRegStats["LFGA3"]

# 3PT Rate
MRegStats["W3Rate"] = MRegStats["WFGA3"] / MRegStats["WFGA"]
MRegStats["L3Rate"] = MRegStats["LFGA3"] / MRegStats["LFGA"]

# FT Perc
#MRegStats["WFTPerc"] = MRegStats["WFTM"] / MRegStats["WFTA"]
#MRegStats["LFTPerc"] = MRegStats["LFTM"] / MRegStats["LFTA"]

# 2PT Perc
MRegStats["WFG2Perc"] = (MRegStats["WFGM"] - MRegStats["WFGM3"]) / (MRegStats["WFGA"] - MRegStats["WFGA3"])
MRegStats["LFG2Perc"] = (MRegStats["LFGM"] - MRegStats["LFGM3"]) / (MRegStats["LFGA"] - MRegStats["LFGA3"])

# 2PT Rate
MRegStats["W2Rate"] = (MRegStats["WFGA"] - MRegStats["WFGA3"]) / MRegStats["WFGA"]
MRegStats["L2Rate"] = (MRegStats["LFGA"] - MRegStats["LFGA3"]) / MRegStats["LFGA"]


In [4]:
# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
MRegStats["WTempo"] = MRegStats["WFGA"] - MRegStats["WOR"] + MRegStats["WTO"] + 0.475 * MRegStats["WFTA"]
MRegStats["LTempo"] = MRegStats["LFGA"] - MRegStats["LOR"] + MRegStats["LTO"] + 0.475 * MRegStats["LFTA"]

# Four Factors
# eFG%
MRegStats["WeFG"] = (MRegStats["WFGM"] + 0.5 * MRegStats["WFGM3"]) / MRegStats["WFGA"]
MRegStats["LeFG"] = (MRegStats["LFGM"] + 0.5 * MRegStats["LFGM3"]) / MRegStats["LFGA"]

# OR%
MRegStats["WORPerc"] = MRegStats["WOR"] / (MRegStats["WOR"] + MRegStats["LDR"])
MRegStats["LORPerc"] = MRegStats["LOR"] / (MRegStats["LOR"] + MRegStats["WDR"])

# TO%
MRegStats["WTOPerc"] = (MRegStats["WTO"] / MRegStats["WTempo"]) * 100
MRegStats["LTOPerc"] = (MRegStats["LTO"] / MRegStats["LTempo"]) * 100

# FTR
MRegStats["WFTR"] = MRegStats["WFTA"] / MRegStats["WFGA"]
MRegStats["LFTR"] = MRegStats["LFTA"] / MRegStats["LFGA"]

In [5]:
# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
MRegStats["WuORTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100
MRegStats["LuORTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
MRegStats["WuDRTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100
MRegStats["LuDRTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
MRegStats["WuNetRtg"] = MRegStats["WuORTG"] - MRegStats["WuDRTG"]
MRegStats["LuNetRtg"] = MRegStats["LuORTG"] - MRegStats["LuDRTG"]


Adjusting the data to be longer instead of wider. Twice as long but groupable by team now.

In [6]:
# Making two dataframes so that they can be added to each other when they change
MWTeamStats = MRegStats.copy()
MLTeamStats = MRegStats.copy()
# Changing location of teams between wins and losses
MLTeamStats["WLoc"] = MLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
MLTeamStats["LLoc"] = MLTeamStats["WLoc"]
MLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for MWTeamStats (Team perspective)
MWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in MWTeamStats.columns if col.startswith("W")}, inplace=True)
MWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in MWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for MLTeamStats (Opponent perspective)
MLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in MLTeamStats.columns if col.startswith("W")}, inplace=True)
MLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in MLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
MTeamStats = pd.concat([MWTeamStats, MLTeamStats], ignore_index=True)

Doing the same but for Tournament Stats

In [7]:
MWTeamStats = MTournStats.copy()
MLTeamStats = MTournStats.copy()
# Changing location of teams between wins and losses
MLTeamStats["WLoc"] = MLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
MLTeamStats["LLoc"] = MLTeamStats["WLoc"]
MLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for MWTeamStats (Team perspective)
MWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in MWTeamStats.columns if col.startswith("W")}, inplace=True)
MWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in MWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for MLTeamStats (Opponent perspective)
MLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in MLTeamStats.columns if col.startswith("W")}, inplace=True)
MLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in MLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
MTournTeamStats = pd.concat([MWTeamStats, MLTeamStats], ignore_index=True)

Additional Stats Added - Conference (Name and Record), Coach, Wins, City (later)

In [8]:
# Merge conference names for conference bias later in the analysis
MTeamStats = MTeamStats.merge(MConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
MTeamStats = MTeamStats.merge(MConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
MTeamStats["ConfGame"] = (MTeamStats["Team_Conf"] == MTeamStats["Opp_Conf"]).astype(int)

MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Opp_uORTG,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame
0,2003,10,1104,68,1328,62,N,0,27,58,...,86.773968,86.773968,90.006618,3.232650,-3.232650,1104,sec,1328,big_twelve,0
1,2003,98,1400,67,1328,61,H,0,25,64,...,95.126706,95.126706,102.290076,7.163371,-7.163371,1400,big_twelve,1328,big_twelve,1
2,2003,124,1400,76,1328,71,A,0,27,50,...,109.356950,109.356950,115.370019,6.013069,-6.013069,1400,big_twelve,1328,big_twelve,1
3,2003,111,1242,70,1328,77,A,0,21,61,...,107.354479,107.354479,96.319229,-11.035249,11.035249,1242,big_twelve,1328,big_twelve,1
4,2003,120,1304,51,1328,76,A,0,21,53,...,108.455227,108.455227,72.237960,-36.217266,36.217266,1304,big_twelve,1328,big_twelve,1


In [9]:
# Add a result to see if a team won or not; this will be used for win percentage later
MTeamStats["Result"] = (MTeamStats["Team_Score"] > MTeamStats["Opp_Score"]).astype(int)

Instead of looking at just net rating, we want to adjust based on the offensive and defensive ratings as well. We normalize the offense and defensive ratings in order to get an adjustment for later.

In [10]:
# Select only numerical columns in the MTeamStats dataframe
TeamStatsNumericCols = MTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
MTeamGroupedStats = MTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
MConfNetStats = MTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
MConfNetStats["Team_uORTGweight"] = MConfNetStats["Team_uORTG"] / MConfNetStats["Team_uORTG"].mean()
MConfNetStats["Team_uDRTGweight"] = MConfNetStats["Team_uDRTG"] / MConfNetStats["Team_uDRTG"].mean()
MConfNetStats["Team_uNetweight"] = (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]) / (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]).mean()

In [11]:
# Merging the conference weights back into the original dataframe
MTeamGroupedStats = MTeamGroupedStats.merge(MConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight","Team_uNetweight"]],on = ["Season","Team_Conf"])

In [12]:
# Creating the conference-adjusted team weights
MTeamGroupedStats["ConfAdjORTG"] = MTeamGroupedStats["Team_uORTG"] * MTeamGroupedStats["Team_uORTGweight"]
MTeamGroupedStats["ConfAdjDRTG"] = MTeamGroupedStats["Team_uDRTG"] * MTeamGroupedStats["Team_uDRTGweight"]
MTeamGroupedStats["ConfAdjustment"] = (MTeamGroupedStats["ConfAdjORTG"] - MTeamGroupedStats["ConfAdjDRTG"]) #* MTeamGroupedStats["Team_uNetweight"]
MTeamGroupedStats["ConfAdjNetRtg"] = MTeamGroupedStats["Team_uNetRtg"] + MTeamGroupedStats["ConfAdjustment"]

In [13]:
MTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjustment"].mean()

Team_TeamID
1101    -6.302463
1102     1.651105
1103     7.152810
1104    13.362988
1105   -17.258504
          ...    
1476   -14.526650
1477   -16.790496
1478   -14.347080
1479   -18.395853
1480   -21.491918
Name: ConfAdjustment, Length: 371, dtype: float64

In [14]:
# Lets add the conference bias onto all of the box scores now
MTeamStats = MTeamStats.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjustment"]], on = ["Season","Team_TeamID"])

In [15]:
MTeamStats["ConfAdjNetRtg"] = MTeamStats["Team_uNetRtg"] + MTeamStats["ConfAdjustment"]
MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result,ConfAdjustment,ConfAdjNetRtg
0,2003,10,1104,68,1328,62,N,0,27,58,...,3.232650,-3.232650,1104,sec,1328,big_twelve,0,1,12.510211,15.742861
1,2003,18,1104,82,1106,56,H,0,24,49,...,37.407372,-37.407372,1104,sec,1106,swac,0,1,12.510211,49.917583
2,2003,21,1104,80,1292,65,H,0,27,59,...,22.745227,-22.745227,1104,sec,1292,sun_belt,0,1,12.510211,35.255437
3,2003,25,1104,54,1326,48,N,0,16,57,...,10.098195,-10.098195,1104,sec,1326,big_ten,0,1,12.510211,22.608405
4,2003,29,1104,89,1422,61,H,0,34,70,...,37.551049,-37.551049,1104,sec,1422,southern,0,1,12.510211,50.061260


In [16]:
# Thank you to ChatGPT for optimizing my code!
# Create an empty list to store processed data
test_list = []

# Group by Season and Team_TeamID to avoid redundant filtering
for (season, team_id), group in MTeamStats.loc[:,["Season","DayNum","Team_TeamID","Opp_TeamID","Team_Score","Opp_Score","ConfAdjNetRtg"]].groupby(["Season", "Team_TeamID"]):
    group = group.sort_values(by = ["Season","DayNum"]).copy()  # Avoid SettingWithCopyWarning
    group["RollingNetRtg"] = group["ConfAdjNetRtg"].expanding().mean()  # Compute rolling mean
    test_list.append(group)  # Store processed group

# Concatenate all processed groups at once (efficient)
MRollStats = pd.concat(test_list, ignore_index=True)

MRollStats.head()


,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,RollingNetRtg
0,2003,19,1102,1257,47,65,-24.648504,-24.648504
1,2003,22,1102,1391,72,43,60.319221,17.835359
2,2003,25,1102,1117,57,52,15.044674,16.905131
3,2003,27,1102,1399,47,60,-23.158939,6.889113
4,2003,31,1102,1410,65,44,53.068828,16.125056


In [17]:
# Get the opponent's stats on that day from the MRollStats dataframe
test = MRollStats.copy()
MRollStats = MRollStats.merge(test.loc[:,["Season","DayNum","Team_TeamID","RollingNetRtg"]], left_on = ["Season","DayNum","Opp_TeamID"], right_on = ["Season","DayNum","Team_TeamID"]).drop("Team_TeamID_y",axis = 1).rename(columns = {"Team_TeamID_x": "Team_TeamID",
                                                                                                                                                                                                                                        "RollingNetRtg_x":"Team_NetRtg",
                                                                                                                                                                                                                                        "RollingNetRtg_y": "Opp_NetRtg"})

In [18]:
MRollStats["Result"] = (MRollStats["Team_Score"] > MRollStats["Opp_Score"]).astype('int')
MRollStats["NetDiff"] = (MRollStats["Team_NetRtg"] - MRollStats["Opp_NetRtg"]).astype('int')
MRollStats["Team_TensRtg"] = (MRollStats["Team_NetRtg"] // 10 * 10).astype('int')
MRollStats["Opp_TensRtg"] = (MRollStats["Opp_NetRtg"] // 10 * 10).astype('int')
MRollStats.head()

,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,Team_NetRtg,Opp_NetRtg,Result,NetDiff,Team_TensRtg,Opp_TensRtg
0,2003,19,1102,1257,47,65,-24.648504,-24.648504,51.307978,0,-75,-30,50
1,2003,22,1102,1391,72,43,60.319221,17.835359,-61.449428,1,79,10,-70
2,2003,25,1102,1117,57,52,15.044674,16.905131,-4.348536,1,21,10,-10
3,2003,27,1102,1399,47,60,-23.158939,6.889113,14.051040,0,-7,0,10
4,2003,31,1102,1410,65,44,53.068828,16.125056,-43.697444,1,59,10,-50


In [19]:
# ok this is my code now

# Two new columms: WTeamID and LTeamID
# Not entirely new but they're now necessary
MRollStats["WTeamID"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Team_TeamID"], MRollStats["Opp_TeamID"])

MRollStats["LTeamID"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Opp_TeamID"], MRollStats["Team_TeamID"])

MRollStats["WNetRtg"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Team_NetRtg"], MRollStats["Opp_NetRtg"])

MRollStats["LNetRtg"] = np.where(MRollStats["Team_Score"] > MRollStats["Opp_Score"],
                                 MRollStats["Opp_NetRtg"], MRollStats["Team_NetRtg"])

MRollStats.sort_values(by = ["Season","DayNum"]).head(5)

,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,Team_NetRtg,Opp_NetRtg,Result,NetDiff,Team_TensRtg,Opp_TensRtg,WTeamID,LTeamID,WNetRtg,LNetRtg
55,2003,10,1104,1328,68,62,15.742861,15.742861,20.899785,1,-5,10,20,1104,1328,15.742861,20.899785
4353,2003,10,1272,1393,70,63,24.731952,24.731952,9.544136,1,15,20,0,1272,1393,24.731952,9.544136
5652,2003,10,1328,1104,62,68,20.899785,20.899785,15.742861,0,5,20,10,1104,1328,15.742861,20.899785
7334,2003,10,1393,1272,63,70,9.544136,9.544136,24.731952,0,-15,0,20,1272,1393,24.731952,9.544136
2165,2003,11,1186,1458,55,81,-35.466989,-35.466989,63.352884,0,-98,-40,60,1458,1186,63.352884,-35.466989


In [20]:
# This was the unoptimized code (it took over 5 minutes to run)

# Due to how long this code takes to run we will not be renaming the dataframe.
#test = pd.DataFrame()
#for i in set(MTeamStats["Season"]):
#    for j in set(MTeamStats["Team_TeamID"]):
#        test1 = MTeamStats[(MTeamStats["Team_TeamID"] == j) & (MTeamStats["Season"] == i)]
#        test1["RollingNetRtg"] = test1["Team_uNetRtg"].expanding().mean() # This creates a rolling mean
#        test = pd.concat([test,test1])
#test1.head()

In [21]:
MRelCompStats = MCompStats[MCompStats["Season"] > 2002] # We only need stats from 2003 onward

# Merge to get the Net Ratings for the Winning and Losing Teams
MRelCompStats = MRelCompStats.merge(MRollStats.loc[:,["Season","DayNum","WTeamID","WNetRtg","LNetRtg"]], on = ["Season","DayNum","WTeamID"])
MRelCompStats.rename(columns = {"RollingNetRtg": "WNetRtg"}, inplace = True)
MRelCompStats.drop_duplicates(inplace=True)

In [22]:
# Create the WDiff column which will end up being our main source of predictions going forward
MRelCompStats["WDiff"] = MRelCompStats["WNetRtg"] - MRelCompStats["LNetRtg"]

MRelCompStats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WNetRtg,LNetRtg,WDiff
0,2003,10,1104,68,1328,62,N,0,15.742861,20.899785,-5.156924
2,2003,10,1272,70,1393,63,N,0,24.731952,9.544136,15.187816
4,2003,11,1266,73,1437,61,N,0,37.818479,-10.047324,47.865803
6,2003,11,1296,56,1457,50,N,0,8.919554,-11.503028,20.422582
8,2003,11,1400,77,1208,71,N,0,29.759609,7.268204,22.491405


Taking a different route, what if we use integers as a range and then calculate the win% of teams whenever they are in a certain integer?

In [ ]:
# Grouping by NetDiff
MNetDiff = MRollStats.groupby(["NetDiff"])["Result"].mean()
MNetDiff.head()

NetDiff
-251    0.0
-236    0.0
-232    0.0
-228    0.0
-226    0.0
Name: Result, dtype: float64

In [224]:
MTestTourney = MTournStats[MTournStats["Season"] == 2024]
MTestTourney = MTestTourney.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","WTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"WNetRtg"})

MTestTourney = MTestTourney.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","LTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"LNetRtg"})

In [225]:
MTestTourney["NetDiff"] = (MTestTourney["WNetRtg"] - MTestTourney["LNetRtg"]).astype('int')
MTestTourney = MTestTourney.merge(MNetDiff,on = ["NetDiff"])

In [226]:
MTestTourney["BrierScore"] = (MTestTourney["Result"] - 1)**2
MTestTourney["BrierScore"].mean()
MTestTourney["CombScore"] = (MTestTourney["WNetRtg"] * MTestTourney["LNetRtg"])**(1/3)

Our initial Brier Score is decent but we can do better - let's bias it using some seeding biases.

In [209]:
MCompSeedStats = MCompTournStats.merge(MSeeds, left_on = ["Season","WTeamID"], right_on = ["Season","TeamID"]).rename(columns = {"Seed":"WSeed"}).merge(MSeeds, left_on = ["Season","LTeamID"], right_on = ["Season","TeamID"]).rename(columns = {"Seed":"LSeed"}).drop(["TeamID_x","TeamID_y"], axis = 1)


In [210]:
def remove_letters(input_string):
  return re.sub(r"[a-zA-Z]", "", input_string)

In [211]:
# Creating wins splits for the seeds both with the region and without the region
MCompSeedStats["WSeedNum"] = [remove_letters(x) for x in MCompSeedStats["WSeed"]]
MCompSeedStats["LSeedNum"] = [remove_letters(x) for x in MCompSeedStats["LSeed"]]
MCompSeedStats["WSeedNum"] = MCompSeedStats["WSeedNum"].astype('int')
MCompSeedStats["LSeedNum"] = MCompSeedStats["LSeedNum"].astype('int')

In [212]:
# Attempting to try and create a bias for the win percentage via the historic seed win percent. This one will need more work.
MSeedWinStats = MCompSeedStats.groupby(["WSeedNum"]).count()
wins = MCompSeedStats["WSeedNum"].value_counts().rename("Wins")
total_games = MCompSeedStats["WSeedNum"].value_counts().add(MCompSeedStats["LSeedNum"].value_counts()).rename("TotalGames")
win_percentage = (wins / total_games).rename("WinPercentage")

Mseed_win_stats = pd.concat([wins, total_games, win_percentage], axis=1).reset_index().rename(columns={"index": "SeedNum"})
Mseed_win_stats["WinPercentagez"] = (Mseed_win_stats["WinPercentage"] - Mseed_win_stats["WinPercentage"].mean())/Mseed_win_stats["WinPercentage"].std()
Mseed_win_stats

,SeedNum,Wins,TotalGames,WinPercentage,WinPercentagez
0,1,515,646,0.797214,1.786919
1,2,363,514,0.706226,1.348252
2,3,287,439,0.653759,1.095300
3,4,243,397,0.612091,0.894413
4,5,180,336,0.535714,0.526191
5,6,163,318,0.512579,0.414651
6,7,139,294,0.472789,0.222820
7,11,122,296,0.412162,-0.069472
8,8,111,266,0.417293,-0.044734
9,9,96,252,0.380952,-0.219939


In [230]:
# Let's try and bias the win percentage based on the combination of Team and Opponent net rating
MCombNet = MRollStats.groupby(["Team_TensRtg","Opp_TensRtg"])["Result"].mean().reset_index()
MTestTourney["WNetTens"] = (MTestTourney["WNetRtg"] // 10 * 10).astype('int')
MTestTourney["LNetTens"] = (MTestTourney["LNetRtg"] // 10 * 10).astype('int')
test = MTestTourney.merge(MCombNet, left_on = ["WNetTens","LNetTens"], right_on = ["Team_TensRtg","Opp_TensRtg"])
test["Final_Result"] = (test["Result_x"] + test["Result_y"])/2
test["BrierScore"] = (test["Final_Result"] - 1)**2
test["BrierScore"].mean()

0.2096853867202486

#### ELO Analysis

In [214]:
# Now lets try and do an ELO analysis to see if that gets us anywhere
def probability(rating1, rating2):
    # Calculate and return the expected score
    return 1.0 / (1 + math.pow(10, (rating1 - rating2) / 400.0))

# Function to calculate Elo rating
# K is a constant.
# outcome determines the outcome: 1 for Player A win, 0 for Player B win, 0.5 for draw.
def elo_rating(Ra, Rb, K, outcome):
    # Calculate the Winning Probability of Player B
    Pb = probability(Ra, Rb)

    # Calculate the Winning Probability of Player A
    Pa = probability(Rb, Ra)

    # Update the Elo Ratings
    Ra = Ra + K * (outcome - Pa)
    Rb = Rb + K * ((1 - outcome) - Pb)

    # Print updated ratings
    return pd.DataFrame({"ELO_A": [Ra],
                         "ELO_B": [Rb]})

# Current ELO ratings
Ra = 1550
Rb = 1450

# K is a constant
K = 100

# Outcome: 1 for Player A win, 0 for Player B win, 0.5 for draw
outcome = 1

# Function call
t = elo_rating(Ra, Rb, K, outcome)
t.iloc[0,1]


1414.0064999802885

In [215]:
MELO = MTeams.copy()
MELO_2000 = MTeams.copy()
MELO_2010 = MTeams.copy()
# Everyone Starts at 1500
MELO["ELO"] = 1500
MELO_2000["ELO"] = 1500
MELO_2010["ELO"] = 1500

In [216]:
#match = MCompStats.loc[0,["WTeamID","LTeamID"]]
# Extract team 1 ELO
#Team1ELO = MELO.loc[MELO["TeamID"] == match.iloc[0].astype('int'), "ELO"]
#Team1ELO

In [217]:
for i in range(0,len(MCompStats)):
    match = MCompStats.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO.loc[MELO["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO.loc[MELO["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO.loc[MELO["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO.loc[MELO["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

C:\Users\mattd\AppData\Local\Temp\ipykernel_31324\4222030709.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1509.9280491829095' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  MELO.loc[MELO["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]


KeyboardInterrupt: 

In [ ]:
MComp_2000 = MCompStats[MCompStats["Season"] > 1999].reset_index()
for i in range(0,len(MComp_2000)):
    match = MComp_2000.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO_2000.loc[MELO_2000["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO_2000.loc[MELO_2000["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO_2000.loc[MELO_2000["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO_2000.loc[MELO_2000["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

In [220]:
MComp_2010 = MCompStats[MCompStats["Season"] > 2009].reset_index()
for i in range(0,len(MComp_2010)):
    match = MComp_2010.loc[i,["WTeamID","LTeamID"]]
    # Extract team 1 ELO
    Team1ELO = MELO_2010.loc[MELO_2010["TeamID"] == match.iloc[0], "ELO"].values[0]
    # Same for team 2
    Team2ELO = MELO_2010.loc[MELO_2010["TeamID"] == match.iloc[1], "ELO"].values[0]

    NewELO = elo_rating(Ra = Team1ELO, Rb = Team2ELO, K = 10, outcome = 1)

    MELO_2010.loc[MELO_2010["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]
    MELO_2010.loc[MELO_2010["TeamID"] == match["LTeamID"],"ELO"] = NewELO.iloc[0,1]

C:\Users\mattd\AppData\Local\Temp\ipykernel_31324\2731285289.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1509.9280491829095' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  MELO_2010.loc[MELO_2010["TeamID"] == match["WTeamID"],"ELO"] = NewELO.iloc[0,0]


In [227]:
MTestTourney = MTestTourney.merge(MELO_2010.loc[:,["TeamID","ELO"]], left_on = ["WTeamID"], right_on = ["TeamID"]).drop("TeamID", axis = 1).rename(columns = {"ELO": "WELO"}).merge(MELO_2010.loc[:,["TeamID","ELO"]], left_on = ["LTeamID"], right_on = ["TeamID"]).drop("TeamID", axis = 1).rename(columns = {"ELO": "LELO"})

In [232]:
MTestTourney["ELOWinPerc"] = [probability(x,y) for (x,y) in zip(MTestTourney["WELO"],MTestTourney["LELO"])]
MTestTourney["BrierScoreELO"] = ((MTestTourney["ELOWinPerc"] + MTestTourney["Result"])/2 - 1)**2
MTestTourney["BrierScoreELO"].mean()

0.26075502616324514

# WOMEN'S ANALYSIS

In [21]:
# Women's regular season detailed stats
WRegStats = pd.read_csv("Data/WRegularSeasonDetailedResults.csv")

# Women regular season compact stats
WCompStats = pd.read_csv("Data/WRegularSeasonCompactResults.csv")

# Women's NCAA tournament detailed stats
WTournStats = pd.read_csv("Data/WNCAATourneyDetailedResults.csv")

# Women's NCAA tournament compact stats
WCompTournStats = pd.read_csv("Data/WNCAATourneyCompactResults.csv")

# Women's Conference tournament stats
WConfTournStats = pd.read_csv("Data/WConferenceTourneyGames.csv")

# Women's Team names
WTeams = pd.read_csv("Data/WTeams.csv")
WTeamSpellings = pd.read_csv("Data/WTeamSpellings.csv", encoding='unicode_escape')

# Women's Tournament Seeds
WSeeds = pd.read_csv("Data/WNCAATourneySeeds.csv")

# Women's Conferences
WConferences = pd.read_csv("Data/WTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
WGameCities = pd.read_csv("Data/WGameCities.csv")

In [22]:
# We don't need to overcomplicate it like we did the men's.
# We really only used the net rating and nothing else so I think that's all we should focus on here

# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
WRegStats["WTempo"] = WRegStats["WFGA"] - WRegStats["WOR"] + WRegStats["WTO"] + 0.475 * WRegStats["WFTA"]
WRegStats["LTempo"] = WRegStats["LFGA"] - WRegStats["LOR"] + WRegStats["LTO"] + 0.475 * WRegStats["LFTA"]

# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
WRegStats["WuORTG"] = (WRegStats["WScore"] / WRegStats["WTempo"]) * 100
WRegStats["LuORTG"] = (WRegStats["LScore"] / WRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
WRegStats["WuDRTG"] = (WRegStats["LScore"] / WRegStats["LTempo"]) * 100
WRegStats["LuDRTG"] = (WRegStats["WScore"] / WRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
WRegStats["WuNetRtg"] = WRegStats["WuORTG"] - WRegStats["WuDRTG"]
WRegStats["LuNetRtg"] = WRegStats["LuORTG"] - WRegStats["LuDRTG"]

In [23]:
# Making two dataframes so that they can be added to each other when they change
WWTeamStats = WRegStats.copy()
WLTeamStats = WRegStats.copy()
# Changing location of teams between wins and losses
WLTeamStats["WLoc"] = WLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
WLTeamStats["LLoc"] = WLTeamStats["WLoc"]
WLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for WWTeamStats (Team perspective)
WWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in WWTeamStats.columns if col.startswith("W")}, inplace=True)
WWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in WWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for WLTeamStats (Opponent perspective)
WLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in WLTeamStats.columns if col.startswith("W")}, inplace=True)
WLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in WLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
WTeamStats = pd.concat([WWTeamStats, WLTeamStats], ignore_index=True)

In [24]:
# Doing the same for tournament stats
WWTeamStats = WTournStats.copy()
WLTeamStats = WTournStats.copy()
# Changing location of teams between wins and losses
WLTeamStats["WLoc"] = WLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
WLTeamStats["LLoc"] = WLTeamStats["WLoc"]
WLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for WWTeamStats (Team perspective)
WWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in WWTeamStats.columns if col.startswith("W")}, inplace=True)
WWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in WWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for WLTeamStats (Opponent perspective)
WLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in WLTeamStats.columns if col.startswith("W")}, inplace=True)
WLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in WLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
WTournTeamStats = pd.concat([WWTeamStats, WLTeamStats], ignore_index=True)

### Adding Conferences to WBB Stats

In [25]:
# Adding Conferences
WTeamStats = WTeamStats.merge(WConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
WTeamStats = WTeamStats.merge(WConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
WTeamStats["ConfGame"] = (WTeamStats["Team_Conf"] == WTeamStats["Opp_Conf"]).astype(int)

# Add a result to see if a team won or not; this will be used for win percentage later
WTeamStats["Result"] = (WTeamStats["Team_Score"] > WTeamStats["Opp_Score"]).astype(int)

WTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result
0,2010,11,3103,63,3237,49,H,0,23,54,...,69.257951,88.701162,19.443211,-19.443211,3103,mac,3237,summit,0,1
1,2010,13,3231,75,3237,52,H,0,26,59,...,70.700204,102.845389,32.145185,-32.145185,3231,big_ten,3237,summit,0,1
2,2010,82,3282,65,3237,36,H,0,26,65,...,53.372869,94.032550,40.659681,-40.659681,3282,summit,3237,summit,1,1
3,2010,112,3282,56,3237,46,A,0,23,54,...,73.925271,88.502568,14.577297,-14.577297,3282,summit,3237,summit,1,1
4,2010,49,3293,88,3237,68,A,0,30,60,...,92.297251,119.039567,26.742316,-26.742316,3293,ovc,3237,summit,0,1


In [28]:
# Select only numerical columns in the WTeamStats dataframe
TeamStatsNumericCols = WTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
WTeamGroupedStats = WTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
WConfNetStats = WTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
WConfNetStats["Team_uORTGweight"] = WConfNetStats["Team_uORTG"] / WConfNetStats["Team_uORTG"].mean()
WConfNetStats["Team_uDRTGweight"] = WConfNetStats["Team_uDRTG"] / WConfNetStats["Team_uDRTG"].mean()
WConfNetStats["Team_uNetweight"] = (WConfNetStats["Team_uORTG"] - WConfNetStats["Team_uDRTG"]) / (WConfNetStats["Team_uORTG"] - WConfNetStats["Team_uDRTG"]).mean()
WTeamGroupedStats = WTeamGroupedStats.merge(WConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight","Team_uNetweight"]],on = ["Season","Team_Conf"])

# Creating the conference-adjusted team weights
WTeamGroupedStats["ConfAdjORTG"] = WTeamGroupedStats["Team_uORTG"] * WTeamGroupedStats["Team_uORTGweight"]
WTeamGroupedStats["ConfAdjDRTG"] = WTeamGroupedStats["Team_uDRTG"] * WTeamGroupedStats["Team_uDRTGweight"]
WTeamGroupedStats["ConfAdjustment"] = (WTeamGroupedStats["ConfAdjORTG"] - WTeamGroupedStats["ConfAdjDRTG"]) #* WTeamGroupedStats["Team_uNetweight"]
WTeamGroupedStats["ConfAdjNetRtg"] = WTeamGroupedStats["Team_uNetRtg"] + WTeamGroupedStats["ConfAdjustment"]

In [29]:
WTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjustment"].mean()

Team_TeamID
3101     0.831935
3102   -14.865361
3103     0.713604
3104    14.193826
3105   -13.762719
          ...    
3476   -22.065787
3477   -13.735953
3478   -26.394570
3479   -22.032496
3480   -10.484587
Name: ConfAdjustment, Length: 366, dtype: float64

In [30]:
# Lets add the conference bias onto all of the box scores now
WTeamStats = WTeamStats.merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjustment"]], on = ["Season","Team_TeamID"])

WTeamStats["ConfAdjNetRtg"] = WTeamStats["Team_uNetRtg"] + WTeamStats["ConfAdjustment"]
WTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result,ConfAdjustment,ConfAdjNetRtg
0,2010,11,3103,63,3237,49,H,0,23,54,...,19.443211,-19.443211,3103,mac,3237,summit,0,1,4.902578,24.345789
1,2010,25,3103,79,3351,43,N,0,28,61,...,49.180510,-49.180510,3103,mac,3351,maac,0,1,4.902578,54.083088
2,2010,30,3103,68,3236,66,H,0,23,58,...,5.931241,-5.931241,3103,mac,3236,summit,0,1,4.902578,10.833819
3,2010,33,3103,75,3464,63,H,0,28,66,...,18.981340,-18.981340,3103,mac,3464,horizon,0,1,4.902578,23.883918
4,2010,45,3103,62,3152,55,A,0,21,55,...,8.927392,-8.927392,3103,mac,3152,gwc,0,1,4.902578,13.829970


In [ ]:
# Thank you to ChatGPT for optimizing my code!
# Create an empty list to store processed data
test_list = []

# Group by Season and Team_TeamID to avoid redundant filtering
for (season, team_id), group in WTeamStats.loc[:,["Season","DayNum","Team_TeamID","Opp_TeamID","Team_Score","Opp_Score","ConfAdjNetRtg"]].groupby(["Season", "Team_TeamID"]):
    group = group.sort_values(by = ["Season","DayNum"]).copy()  # Avoid SettingWithCopyWarning
    group["RollingNetRtg"] = group["ConfAdjNetRtg"].expanding().mean()  # Compute rolling mean
    test_list.append(group)  # Store processed group

# Concatenate all processed groups at once (efficient)
WRollStats = pd.concat(test_list, ignore_index=True)

# Get the opponent's stats on that day from the WRollStats dataframe
test = WRollStats.copy()
WRollStats = WRollStats.merge(test.loc[:,["Season","DayNum","Team_TeamID","RollingNetRtg"]], left_on = ["Season","DayNum","Opp_TeamID"], right_on = ["Season","DayNum","Team_TeamID"]).drop("Team_TeamID_y",axis = 1).rename(columns = {"Team_TeamID_x": "Team_TeamID",
                                                                                                                                                                                                                                        "RollingNetRtg_x":"Team_NetRtg",
                                                                                                                                                                                                                                        "RollingNetRtg_y": "Opp_NetRtg"})
WRollStats["Result"] = (WRollStats["Team_Score"] > WRollStats["Opp_Score"]).astype('int')
WRollStats["NetDiff"] = (WRollStats["Team_NetRtg"] - WRollStats["Opp_NetRtg"]).astype('int')
WRollStats["Team_TensRtg"] = (WRollStats["Team_NetRtg"] // 10 * 10).astype('int')
WRollStats["Opp_TensRtg"] = (WRollStats["Opp_NetRtg"] // 10 * 10).astype('int')
WRollStats.head()


,Season,DayNum,Team_TeamID,Opp_TeamID,Team_Score,Opp_Score,ConfAdjNetRtg,Team_NetRtg,Opp_NetRtg,Result,NetDiff,Team_TensRtg,Opp_TensRtg
0,2010,11,3102,3394,46,65,-54.785714,-54.785714,29.459539,0,-84,-60,20
1,2010,12,3102,3399,49,81,-74.025743,-64.405729,12.705601,0,-77,-70,10
2,2010,18,3102,3339,65,73,-42.185927,-56.999128,2.797434,0,-59,-60,0
3,2010,23,3102,3119,42,60,-57.748541,-57.186481,-11.056774,0,-46,-60,-20
4,2010,25,3102,3392,60,72,-43.754422,-54.500070,-33.283258,0,-21,-60,-40


Let's try something a little different - what if instead of using the difference in net rating, we instead use the difference in rank? This could be very wonky but we don't know unless we try!

Not every team played on every day, so we are going to have to go by week - or even by every 10 days just to make sure this isn't weird.

In [107]:
# Creating a rank stats dataframe
WRankStats = WRollStats[["Season","DayNum","Team_TeamID","Team_NetRtg","Result"]]

# The "DayTens" column allows us to make sure that almost every team is included in every ranking
# This will create a few oddities but nothing too substantial 
WRankStats["DayTens"] = WRankStats["DayNum"] // 10 * 10
WRankStats = (WRankStats.groupby(["Season","DayTens","Team_TeamID"])[["Team_NetRtg","Result"]]
 .last()
 .reset_index())

# I did this without thinking but turns out you can create a column using a groupby function, the more you know
WRankStats["Rank"] = WRankStats.groupby(["Season","DayTens"])[["Team_NetRtg"]].rank()
WRankStats.tail()

C:\Users\mattd\AppData\Local\Temp\ipykernel_25836\1832166340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WRankStats["DayTens"] = WRankStats["DayNum"] // 10 * 10


,Season,DayTens,Team_TeamID,Team_NetRtg,Result,Rank
67741,2025,100,3476,-25.125060,1,77.0
67742,2025,100,3477,-31.740101,0,54.0
67743,2025,100,3478,-67.468491,0,4.0
67744,2025,100,3479,-31.039714,0,57.0
67745,2025,100,3480,-18.361824,1,102.0


In [32]:
# ok this is my code now

# Two new columms: WTeamID and LTeamID
# Not entirely new but they're now necessary
WRollStats["WTeamID"] = np.where(WRollStats["Team_Score"] > WRollStats["Opp_Score"],
                                 WRollStats["Team_TeamID"], WRollStats["Opp_TeamID"])

WRollStats["LTeamID"] = np.where(WRollStats["Team_Score"] > WRollStats["Opp_Score"],
                                 WRollStats["Opp_TeamID"], WRollStats["Team_TeamID"])

WRollStats["WNetRtg"] = np.where(WRollStats["Team_Score"] > WRollStats["Opp_Score"],
                                 WRollStats["Team_NetRtg"], WRollStats["Opp_NetRtg"])

WRollStats["LNetRtg"] = np.where(WRollStats["Team_Score"] > WRollStats["Opp_Score"],
                                 WRollStats["Opp_NetRtg"], WRollStats["Team_NetRtg"])

WRollStats.sort_values(by = ["Season","DayNum"]).head(5)

WRelCompStats = WCompStats[WCompStats["Season"] > 2002] # We only need stats from 2003 onward

# Werge to get the Net Ratings for the Winning and Losing Teams
WRelCompStats = WRelCompStats.merge(WRollStats.loc[:,["Season","DayNum","WTeamID","WNetRtg","LNetRtg"]], on = ["Season","DayNum","WTeamID"])
WRelCompStats.rename(columns = {"RollingNetRtg": "WNetRtg"}, inplace = True)
WRelCompStats.drop_duplicates(inplace=True)

# Create the WDiff column which will end up being our main source of predictions going forward
WRelCompStats["WDiff"] = WRelCompStats["WNetRtg"] - WRelCompStats["LNetRtg"]

WRelCompStats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WNetRtg,LNetRtg,WDiff
0,2010,11,3103,63,3237,49,H,0,24.345789,-49.794907,74.140696
2,2010,11,3104,73,3399,68,N,0,13.837261,-15.025510,28.862771
4,2010,11,3110,71,3224,59,A,0,25.286564,-26.261320,51.547884
6,2010,11,3111,63,3267,58,A,0,5.531480,-6.298573,11.830053
8,2010,11,3119,74,3447,70,H,1,-4.575289,-16.818148,12.242859


Now we're going to try and see if the Rank Difference makes any difference at all

In [ ]:
WDayRankStats = WTeamStats.copy().drop(WTeamStats.loc[:,"Team_Loc":].columns.tolist(), axis = 1)
WDayRankStats["DayTens"] = WDayRankStats["DayNum"] // 10 * 10

WDayRankStats = (WDayRankStats.merge(WRankStats, on = ["Season","DayTens","Team_TeamID"])
 .merge(WRankStats, left_on = ["Season","DayTens","Opp_TeamID"], right_on = ["Season","DayTens","Team_TeamID"])
 .drop(["Result_y","Team_NetRtg_x","Team_NetRtg_y","Team_TeamID_y","Result_x"], axis = 1)
 .rename(columns = {"Team_TeamID_x": "Team_TeamID",
                    "Rank_x":"Team_Rank",
                    "Rank_y": "Opp_Rank"}))

WDayRankStats["RankDif"] = WDayRankStats["Team_Rank"] - WDayRankStats["Opp_Rank"]
WDayRankStats["Result"] = (WDayRankStats["Team_Score"] > WDayRankStats["Opp_Score"]).astype('int')
WDayRankStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,DayTens,Team_Rank,Opp_Rank,RankDif,Result
0,2010,11,3103,63,3237,49,10,127.0,12.0,115.0,1
1,2010,13,3231,75,3237,52,10,260.0,12.0,248.0,1
2,2010,17,3296,71,3237,55,10,202.0,12.0,190.0,1
3,2010,13,3103,37,3382,67,10,127.0,318.0,-191.0,0
4,2010,15,3145,47,3382,65,10,88.0,318.0,-230.0,0


In [128]:
# Max value of rank
WRankMaxTourn = WTeamGroupedStats.groupby(["Season","Team_TeamID"])[["ConfAdjNetRtg"]].mean().reset_index()
WRankMaxTourn["Rank"] = WRankMaxTourn.groupby(["Season"])[["ConfAdjNetRtg"]].rank()
WRankMaxTourn.head()

,Season,Team_TeamID,ConfAdjNetRtg,Rank
0,2010,3102,-58.167200,4.0
1,2010,3103,9.109104,225.0
2,2010,3104,0.994098,181.0
3,2010,3105,-15.196832,93.0
4,2010,3106,-10.978179,116.0


In [119]:
WRankDif = (WDayRankStats.groupby(["RankDif"])[["Result"]]
            .mean()
            .reset_index())
WRankDif

,RankDif,Result
0,-358.0,0.0
1,-352.0,0.0
2,-351.0,0.0
3,-350.0,0.0
4,-349.0,0.0
...,...,...
699,349.0,1.0
700,350.0,1.0
701,351.0,1.0
702,352.0,1.0


In [144]:
WTestTourneyRank = WCompTournStats[WCompTournStats["Season"] == 2024]
WTestTourneyRank["DayTens"] = WTestTourneyRank["DayNum"] // 10 * 10
WTestTourneyRank = (WTestTourneyRank.merge(WRankMaxTourn.loc[:,["Season","Team_TeamID","Rank"]], 
                   left_on = ["Season","WTeamID"], 
                   right_on = ["Season","Team_TeamID"])
                   .rename(columns = {"Rank":"WRank"})
                   .drop(["Team_TeamID"], axis = 1))

WTestTourneyRank = (WTestTourneyRank.merge(WRankMaxTourn.loc[:,["Season","Team_TeamID","Rank"]], 
                   left_on = ["Season","LTeamID"], 
                   right_on = ["Season","Team_TeamID"])
                   .rename(columns = {"Rank": "LRank"})
                   .drop(["Team_TeamID"], axis = 1))

WTestTourneyRank["RankDif"] = (WTestTourneyRank["WRank"] - WTestTourneyRank["LRank"])
WTestTourneyRank = WTestTourneyRank.merge(WRankDif,on = ["RankDif"])
WTestTourneyRank.head()

C:\Users\mattd\AppData\Local\Temp\ipykernel_25836\2955094843.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WTestTourneyRank["DayTens"] = WTestTourneyRank["DayNum"] // 10 * 10


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,DayTens,WRank,LRank,RankDif,Result
0,2024,135,3342,49,3357,42,N,0,130,113.0,243.0,-130.0,0.144737
1,2024,135,3435,72,3162,68,N,0,130,301.0,322.0,-21.0,0.356863
2,2024,136,3112,69,3120,59,N,0,130,284.0,304.0,-20.0,0.368142
3,2024,136,3221,72,3404,45,N,0,130,257.0,164.0,93.0,0.798429
4,2024,137,3104,82,3199,74,N,0,130,337.0,297.0,40.0,0.666667


In [145]:
WTestTourneyRank["BrierScore"] = (WTestTourneyRank["Result"] - 1)**2
WTestTourneyRank["BrierScore"].mean()

0.19281036174325486

Now we're using the same method as the men's: Using the integer difference in net rating to try and predict the win% of future net rating differences.

In [33]:
# Grouping by NetDiff
WNetDiff = WRollStats.groupby(["NetDiff"])["Result"].mean()

In [153]:
WTestTourney = WTournStats[WTournStats["Season"] == 2014]
WTestTourney = WTestTourney.merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","WTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"WNetRtg"})

WTestTourney = WTestTourney.merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]], 
                   left_on = ["Season","LTeamID"], 
                   right_on = ["Season","Team_TeamID"]).drop("Team_TeamID",axis = 1).rename(columns = {"ConfAdjNetRtg":"LNetRtg"})

WTestTourney["NetDiff"] = (WTestTourney["WNetRtg"] - WTestTourney["LNetRtg"]).astype('int')
WTestTourney = WTestTourney.merge(WNetDiff,on = ["NetDiff"])

In [154]:

WTestTourney["BrierScore"] = (WTestTourney["Result"] - 1)**2
WTestTourney["BrierScore"].mean() 
#WTestTourney["CombScore"] = (WTestTourney["WNetRtg"] * WTestTourney["LNetRtg"])**(1/3)

0.16623028272790577